In [2]:
import numpy as np

In [71]:
X = np.random.randint( low = 0, high = 2, size =100 )

In [46]:
#result = np.where(X == 1)[0]

In [84]:
ikluder_alle = np.where(y == 1)
velg_noen = np.where(y == 0)#[:len(ikluder_alle[0])]

In [85]:
ikluder_alle

(array([ 0,  1,  2,  4,  5,  7,  8, 10, 11, 14, 15, 18, 21, 22, 23, 24, 26,
        27, 28, 29, 30, 31, 34, 35, 37, 39, 40, 41, 45, 46, 48, 53, 55, 57,
        58, 61, 62, 63, 66, 67, 68, 71, 73, 74, 76, 80, 81, 82, 83, 86, 87,
        88, 89, 94, 96, 97, 98, 99]),)

In [87]:
velg_noen[0][:len(ikluder_alle[0])]

array([ 3,  6,  9, 12, 13, 16, 17, 19, 20, 25, 32, 33, 36, 38, 42, 43, 44,
       47, 49, 50, 51, 52, 54, 56, 59, 60, 64, 65, 69, 70, 72, 75, 77, 78,
       79, 84, 85, 90, 91, 92, 93, 95])

In [93]:
X_sample_idx = np.concatenate( (ikluder_alle[0], velg_noen[0][:len(ikluder_alle[0])] ))

array([ 0,  1,  2,  4,  5,  7,  8, 10, 11, 14, 15, 18, 21, 22, 23, 24, 26,
       27, 28, 29, 30, 31, 34, 35, 37, 39, 40, 41, 45, 46, 48, 53, 55, 57,
       58, 61, 62, 63, 66, 67, 68, 71, 73, 74, 76, 80, 81, 82, 83, 86, 87,
       88, 89, 94, 96, 97, 98, 99,  3,  6,  9, 12, 13, 16, 17, 19, 20, 25,
       32, 33, 36, 38, 42, 43, 44, 47, 49, 50, 51, 52, 54, 56, 59, 60, 64,
       65, 69, 70, 72, 75, 77, 78, 79, 84, 85, 90, 91, 92, 93, 95])

In [94]:
Xtrain = Xtrain[samples_idx]
ytrain = ytrain[samples_idx]

NameError: name 'Xtrain' is not defined

In [16]:
n_samples, n_cols = X.shape

In [17]:
idx = np.arange(n_samples)

In [19]:
np.random.shuffle(idx)

In [21]:
batch_size = 20

In [25]:
splits = int(n_samples/batch_size)

In [26]:
np.array_split(idx, splits)

[array([55, 83, 61, 33, 67, 63,  3, 69, 84,  0, 65, 94, 13, 82, 87, 78,  9,
        89, 71, 64]),
 array([35, 20, 47,  5, 15, 25, 40, 28,  7, 42, 98, 32, 29, 97, 59, 95, 23,
        22, 60, 54]),
 array([72, 27, 92, 75, 81, 26, 57, 24, 58, 53, 66, 85, 74, 50, 79, 19, 91,
         4, 14, 76]),
 array([90, 93, 34, 62, 45,  6, 68, 21, 48, 37,  2,  8, 46, 41, 44, 99, 86,
        11, 56, 70]),
 array([77, 36, 88, 10, 43, 16, 38, 80, 96, 51,  1, 12, 31, 30, 73, 39, 17,
        18, 52, 49])]

In [28]:
for i  in range(epochs):
    np.random.shuffle(X)
    for b_idx in np.array_split(idx, splits):
        print(X[b_idx].shape)

(20, 20)
(20, 20)
(20, 20)
(20, 20)
(20, 20)


In [ ]:
y[:, np.newaxis]

In [108]:
a = np.random.rand(100,50)
b = np.random.rand( 100, 1)

In [109]:
a.shape, b.shape

((100, 50), (100, 1))

In [110]:
a*b

array([[0.23323748, 0.44538605, 0.66647726, ..., 0.630648  , 0.56451252,
        0.48303946],
       [0.0690281 , 0.41039046, 0.09308514, ..., 0.32881093, 0.48927698,
        0.53865895],
       [0.19166383, 0.21622818, 0.20625896, ..., 0.06909733, 0.18876802,
        0.06330485],
       ...,
       [0.5236544 , 0.59996971, 0.02558946, ..., 0.69740488, 0.38059969,
        0.30574931],
       [0.09863614, 0.10334819, 0.00577015, ..., 0.04006364, 0.01812493,
        0.0799606 ],
       [0.10711582, 0.01351512, 0.25895138, ..., 0.00790867, 0.04411055,
        0.09249486]])

In [99]:
(a+b).shape

(100, 50)

In [101]:
np.zeros((1,29))

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])